# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import nltk
nltk.download(['punkt','wordnet','stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
#from custom_transformer import StartingVerbExtractor

In [14]:
# Sample data for faster processing
filename = 'sqlite:///InsertDatabaseName.db'
def load_data(filename):
    engine = create_engine(filename)
    df = pd.read_sql_table('InsertTableName',con=engine)
    X = df.iloc[:100,1]
    Y = df.iloc[:100,4:6]
    return X, Y

In [21]:
#Load data from database
filename = 'sqlite:///InsertDatabaseName.db'
def load_data(filename):
    '''Loading files from database
    and returning target and input values'''
    engine = create_engine(filename)
    df = pd.read_sql_table('InsertTableName',con=engine)
    X = df['message']
    Y = df.drop(['id', 'message', 'original', 'genre'],  axis=1).astype(float)
    categories = Y.columns.values
    return X, Y, categories
X, Y, Category = load_data(filename)

In [26]:
Category = Y.columns.values
cat_counts = [sum(Y[x]) for x in Category]
cat_counts

[20252.0,
 4464.0,
 118.0,
 10841.0,
 2081.0,
 1311.0,
 724.0,
 471.0,
 859.0,
 0.0,
 1669.0,
 2917.0,
 2308.0,
 404.0,
 603.0,
 298.0,
 874.0,
 1192.0,
 3441.0,
 1705.0,
 1199.0,
 1331.0,
 532.0,
 159.0,
 283.0,
 120.0,
 309.0,
 1151.0,
 7286.0,
 2149.0,
 2440.0,
 282.0,
 2452.0,
 528.0,
 1376.0,
 5064.0]

### 2. Write a tokenization function to process your text data

In [15]:
def tokenize(text):
    '''Tokenize the message and remove stop words
    for best performance of ML algorithm
    '''
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-1]", " ", text)
    
    #Tokenize
    tokens = word_tokenize(text)
    #Stopword
    words = [w for w in tokens if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
def model_pipeline():
    '''Multi Output RandomForest model
    along with features - declared as a pipeline'''
    pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=-1))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [17]:
#Load the data
X, y = load_data(filename)
#Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
#Train the model
model = model_pipeline()
model.fit(X_train, y_train)
#Predict the model
y_pred = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
#Preparing dataframe of predictions
target_names = y_test.columns
y_test1 = y_test.reset_index(drop=True).astype(int)
y_pred1 = pd.DataFrame(data = y_pred, columns = y_test.columns).astype(int)

#Iterating through the columns to get the classification report
for item in target_names:
    print(item)
    print(classification_report(y_test1[item], y_pred1[item], target_names=item))

related
             precision    recall  f1-score   support

          r       0.00      0.00      0.00         8
          e       0.68      1.00      0.81        17

avg / total       0.46      0.68      0.55        25

request
             precision    recall  f1-score   support

          r       0.73      0.92      0.81        12
          e       0.90      0.69      0.78        13

avg / total       0.82      0.80      0.80        25



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
def model_withgrid():
    '''Model with grid search to find the most efficient
    paramenter - Lesser number of Parameter given for grid search
    for faster processing'''
    pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier(),n_jobs=-1))
                ])
    
    #Lesser parameters given to save processing time
    parameters =  {'vect__ngram_range': ((1, 1), (1, 2)),
                   #'tfidf__use_idf': (True, False),
                   #'clf__estimator__min_samples_split': [2, 3, 4]
                   }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [32]:
def display_results(cv, y_test, y_pred):
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
def main():
    '''Main function loads data, splits and trains model
    Finally prints the best params'''
    X, y = load_data(filename)

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = model_withgrid()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("\nGrid Score:", model.grid_scores_)
    print("\nBest Parameters:", model.best_params_)
    
main()


Grid Score: [mean: 0.57333, std: 0.08219, params: {'vect__ngram_range': (1, 1)}, mean: 0.53333, std: 0.16438, params: {'vect__ngram_range': (1, 2)}]

Best Parameters: {'vect__ngram_range': (1, 1)}


/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [31]:
#Uses Logistics regression
def model_pipeline():
    '''Uses logistics regression as an alternative to 
    RandomForest'''
    pipeline = Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(LogisticRegression(),n_jobs=-1))
    ])

    return pipeline

def main():
    X, y = load_data(filename)

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = model_pipeline()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

main()

array([[1, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 0],
       [1, 1],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 1]])

### 9. Export your model as a pickle file

In [25]:
#creates pickle file
m = pickle.dumps('clf')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.